In [27]:


import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
    target_transform = None
)
train_data.classes


['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [4]:
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),

)

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_data,
    batch_size = 32,
    shuffle= True
)




In [6]:
test_dataloader = DataLoader(
    test_data,
    batch_size = 32,
    shuffle = True
)

In [7]:
print(f"len of train dataloader {len(train_dataloader)}")
print(f"len of test dataloader {len(test_dataloader)}")

len of train dataloader 1875
len of test dataloader 313


In [30]:
import torch.nn.functional as F
class CNNClassifier(nn.Module):
  def __init__(self,input_shape,hidden_units,output_shape):
    super().__init__()

    self.layer1 = nn.Sequential(
      nn.Conv2d(in_channels = input_shape,out_channels = hidden_units,kernel_size = 3,stride = 1,padding = 1),
      nn.ReLU(),
      nn.Conv2d(in_channels = hidden_units,out_channels = hidden_units,kernel_size = 3,stride = 1,padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_units,out_channels = hidden_units,kernel_size = 3,stride = 1,padding = 1),
      nn.ReLU(),
      nn.Conv2d(in_channels = hidden_units,out_channels = hidden_units,kernel_size = 3,stride = 1,padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*7*7,out_features = output_shape)
    )
  def forward(self, x):
    x = self.layer1(x)
    print(f"After layer1: {x.shape}")

    x = self.layer2(x)
    print(f"After layer2: {x.shape}")

    x = self.classifier(x)
    print(f"After classifier: {x.shape}")

    return x


CNN = CNNClassifier(input_shape = 1,hidden_units = 10,output_shape = 10)



In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params =CNN.parameters(),lr = 0.1 )

In [32]:
def acc_fn(Y_pred,Y_test):
  _,Y_pred = torch.max(Y_pred,1)
  return ((Y_pred==Y_test).sum())/len(Y_pred)

In [33]:

def trainer(dataloader,model,acc_fn,loss_fn,optimizer):
  train_loss = 0.0
  train_accuracy = 0.0
  model.train()
  for batch,(X,y) in enumerate(dataloader):

    y_pred = model(X)
    loss = loss_fn(y_pred,y)
    train_loss+=loss.item()
    train_accuracy +=acc_fn(y_pred,y).item()


    optimizer.zero_grad()
    loss.backward()

    optimizer.step()
  train_loss = (train_loss)/len(dataloader)
  train_accuracy = (train_accuracy)/len(dataloader)
  print(f"Training Loss: {train_loss}|Training Accuracy : {train_accuracy}")


def tester(dataloader,model,acc_fn,loss_fn):

  test_loss = 0.0
  test_accuracy = 0.0

  model.eval()
  with torch.inference_mode():
    for X,y in (dataloader):

      y_pred = model(X)

      loss = loss_fn(y_pred,y)
      test_loss+=loss.item()
      test_accuracy +=acc_fn(y_pred,y).item()

    test_loss = (test_loss)/len(dataloader)
    test_accuracy = (test_accuracy)/len(dataloader)
    print(f"Testing Loss: {test_loss}|Testing Accuracy : {test_accuracy}\n")




In [ ]:
for epoch in range(5):
  print(f"\nEpoch :{epoch}\n__________________________________")
  trainer(train_dataloader,CNN,acc_fn,loss_fn,optimizer)
  tester(test_dataloader,CNN,acc_fn,loss_fn)


In [37]:
from PIL import Image
from torchvision import transforms

path = "/test_data_for_cnn.png"
image_dim = Image.open(path)
image = image_dim.convert("L")
image = image.resize((28,28))

transform = transforms.Compose([transforms.ToTensor()])
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)
image_tensor.shape

torch.Size([1, 1, 28, 28])

In [39]:
def predict(CNN,X):
    pred_class_idx = torch.argmax(CNN.forward(X))

    class_names = ['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']
    return class_names[pred_class_idx.item()]
predict(CNN,image_tensor)

After layer1: torch.Size([1, 10, 14, 14])
After layer2: torch.Size([1, 10, 7, 7])
After classifier: torch.Size([1, 10])


'Bag'